In [1]:
DATA_NAME = 'coco-indoor' 
TRANSFORM = 'wavelet-horizontal'
CHANNEL = 'green'

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_green_horizontal_wavelet_indoor_coco.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
        cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8,9
2,47880.97,0.89,0.22,0.15,0.01,0.0,-0.00,-0.00
3,0.89,9662.85,-0.10,0.05,-0.00,0.0,-0.00,0.00
4,0.22,-0.10,1836.00,0.01,0.00,-0.0,0.00,-0.00
5,0.15,0.05,0.01,288.49,0.00,0.0,0.00,-0.00
6,0.01,-0.00,0.00,0.00,42.02,0.0,0.00,0.00
7,0.00,0.00,-0.00,0.00,0.00,5.6,-0.00,0.00
8,-0.00,-0.00,0.00,0.00,0.00,-0.0,0.71,0.00
9,-0.00,0.00,-0.00,-0.00,0.00,0.0,0.00,0.08


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,2,3,4,5,6,7,8,9
2,1.00000,0.00004,0.00002,0.00004,0.00001,0.00000,-0.00000,-0.00001
3,0.00004,1.00000,-0.00002,0.00003,-0.00000,0.00001,-0.00003,0.00003
4,0.00002,-0.00002,1.00000,0.00002,0.00001,-0.00004,0.00004,-0.00002
5,0.00004,0.00003,0.00002,1.00000,0.00002,0.00001,0.00001,-0.00002
6,0.00001,-0.00000,0.00001,0.00002,1.00000,0.00002,0.00002,0.00002
7,0.00000,0.00001,-0.00004,0.00001,0.00002,1.00000,-0.00003,0.00001
8,-0.00000,-0.00003,0.00004,0.00001,0.00002,-0.00003,1.00000,0.00002
9,-0.00001,0.00003,-0.00002,-0.00002,0.00002,0.00001,0.00002,1.00000


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

1.321383366571056

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[2.86827679e+08 1.14356343e+07 4.01279978e+05 9.51472370e+03
 1.89299429e+02 2.99698590e+00 3.61909102e-02 2.35890560e-34]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7
0,0.999535,-0.029983,-0.005440,-0.000851,-0.000124,-0.000017,-0.000002,-2.372385e-07
1,0.029783,0.998999,-0.032990,-0.004963,-0.000723,-0.000096,-0.000012,-1.223650e-06
2,0.006388,0.032632,0.998913,-0.032349,-0.004575,-0.000612,-0.000076,-8.531565e-06
3,0.001198,0.005954,0.031969,0.998763,-0.037288,-0.004842,-0.000612,-6.682167e-05
4,0.000218,0.001083,0.005701,0.036856,0.998259,-0.045324,-0.005599,-6.073443e-04
5,0.000039,0.000192,0.001012,0.006427,0.044654,0.996891,-0.064228,-6.806439e-03
6,0.000007,0.000036,0.000189,0.001209,0.008304,0.062796,0.992066,-1.085886e-01
7,0.000002,0.000008,0.000042,0.000265,0.001820,0.013657,0.107927,9.940633e-01


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist = cos_dist = [np.min(row) for row in cos_dist]
cos_dist


[0.0004647579154528625,
 0.0010007517129108079,
 0.0010873290713210304,
 0.0012373011001500789,
 0.0017405403213814896,
 0.0031087864304245594,
 0.007934102098400952,
 0.005936713775191005]